In [6]:
import numpy as np
import yfinance as yf
import pandas as pd
from datetime import datetime
import pytz

In [7]:
# ==============================================================================
# 1. DATA HELPER FUNCTIONS (Your Existing Code)
# ==============================================================================

def get_market_data(ticker_symbol):
    """
    Fetches spot price, option chains, and historical data.
    """
    stock = yf.Ticker(ticker_symbol)
    
    # robust fetch for current price
    try:
        current_price = stock.history(period='1d')['Close'].iloc[-1]
    except:
        current_price = stock.info.get('regularMarketPrice', 0.0)

    historical_df = stock.history(period="1y")
    
    # Fetch option chains
    expiration_dates = stock.options
    all_calls_list = []
    
    # Limit to first 3 expiries to save time during testing
    print(f"Fetching option chains for {ticker_symbol}...")
    for expiry in expiration_dates[:3]: 
        try:
            option_chain = stock.option_chain(expiry)
            calls = option_chain.calls
            calls['expirationDate'] = expiry
            all_calls_list.append(calls)
        except Exception as e:
            print(f"Skipping expiry {expiry}: {e}")
    
    if all_calls_list:
        combined_calls_df = pd.concat(all_calls_list, ignore_index=True)
    else:
        combined_calls_df = pd.DataFrame()

    return {
        'spot_price': current_price,
        'calls_df': combined_calls_df,
        'historical_df': historical_df
    }

def calculate_historical_volatility(data_frame):
    """
    Calculates annualized volatility from 1 year of daily returns.
    """
    # Log returns: ln(Today / Yesterday)
    data_frame['log_ret'] = np.log(data_frame['Close'] / data_frame['Close'].shift(1))
    daily_vol = data_frame['log_ret'].std()
    
    # Annualize (x sqrt(252))
    return daily_vol * np.sqrt(252)

def get_risk_free_rate():
    """
    Fetches the 13-week Treasury Bill Rate (^IRX).
    """
    try:
        irx = yf.Ticker("^IRX")
        return irx.history(period="1d")['Close'].iloc[-1] / 100
    except:
        print("Warning: Could not fetch risk-free rate, defaulting to 4.5%")
        return 0.045

def calculate_time_to_expiry(expiry_string):
    """
    Converts expiry string (YYYY-MM-DD) to years (T).
    """
    expiry_date = datetime.strptime(expiry_string, '%Y-%m-%d').date()
    today = datetime.now(pytz.timezone('America/New_York')).date()
    
    days_diff = (expiry_date - today).days
    return max(days_diff / 365.0, 1/365.0) # Avoid division by zero

# Derivation of Stock Price Formula under Geometric Brownian Motion

---

## 1. Assumption: Stock Price Dynamics

We assume that the stock price follows **Geometric Brownian Motion (GBM)**:

$$
dS_t = \mu S_t\,dt + \sigma S_t\,dW_t
$$

where:
- $S_t$ = stock price at time $t$
- $\mu$ = drift (mean rate of return)
- $\sigma$ = volatility
- $W_t$ = Wiener process (Brownian motion)
- $dW_t \sim \mathcal{N}(0, dt)$

The equation has:
- a **deterministic component**: $\mu S_t\,dt$
- a **random component**: $\sigma S_t\,dW_t$

---

## 2. Motivation for Log Transformation

The stock price appears multiplicatively with randomness, making direct integration difficult.

To simplify, define:

$$
f(S_t) = \ln S_t
$$

Logarithms convert **multiplicative growth into additive growth**, which is easier to handle.

---

## 3. Itô’s Lemma

For a twice-differentiable function $f(S_t)$, Itô’s Lemma states:

$$
df = f'(S_t)\,dS_t + \frac{1}{2} f''(S_t)(dS_t)^2
$$

This differs from ordinary calculus due to the presence of stochastic terms.

---

## 4. Compute Derivatives

For $f(S) = \ln S$:

$$
f'(S) = \frac{1}{S},
\qquad
f''(S) = -\frac{1}{S^2}
$$

---

## 5. Compute $(dS_t)^2$

Given:

$$
dS_t = \mu S_t\,dt + \sigma S_t\,dW_t
$$

Square both sides:

$$
(dS_t)^2 = (\mu S_t\,dt + \sigma S_t\,dW_t)^2
$$

Expand:

$$
(dS_t)^2
= (\mu S_t\,dt)^2
+ 2(\mu S_t\,dt)(\sigma S_t\,dW_t)
+ (\sigma S_t\,dW_t)^2
$$

Using Itô rules:
- $dt^2 = 0$
- $dt\,dW_t = 0$
- $(dW_t)^2 = dt$

Only the last term survives:

$$
(dS_t)^2 = \sigma^2 S_t^2\,dt
$$

---

## 6. Apply Itô’s Lemma

Substitute into Itô’s formula:

$$
d(\ln S_t)
= \frac{1}{S_t} dS_t
- \frac{1}{2} \frac{1}{S_t^2} (dS_t)^2
$$

Substitute $dS_t$ and $(dS_t)^2$:

$$
d(\ln S_t)
= \frac{1}{S_t}(\mu S_t\,dt + \sigma S_t\,dW_t)
- \frac{1}{2} \frac{1}{S_t^2}(\sigma^2 S_t^2\,dt)
$$

Simplify:

$$
d(\ln S_t)
= \mu\,dt + \sigma\,dW_t - \frac{1}{2}\sigma^2\,dt
$$

$$
\boxed{
d(\ln S_t)
= \left(\mu - \frac{1}{2}\sigma^2\right)dt + \sigma\,dW_t
}
$$

---

## 7. Integration Over Time

Integrate from $t=0$ to $t=T$:

$$
\ln S_T - \ln S_0
= \left(\mu - \frac{1}{2}\sigma^2\right)T + \sigma W_T
$$

Rearrange:

$$
\ln S_T
= \ln S_0
+ \left(\mu - \frac{1}{2}\sigma^2\right)T
+ \sigma W_T
$$

---

## 8. Solve for $S_T$

Exponentiate both sides:

$$
\boxed{
S_T
= S_0
\exp\left[
\left(\mu - \frac{1}{2}\sigma^2\right)T
+ \sigma W_T
\right]
}
$$

---

## 9. Risk-Neutral Measure

For derivative pricing, the real drift $\mu$ is replaced by the risk-free rate $r$:

$$
\mu \rightarrow r
$$

---

## 10. Monte Carlo Representation

Since Brownian motion satisfies:

$$
W_T \sim \sqrt{T}\,Z,
\qquad
Z \sim \mathcal{N}(0,1)
$$

the final simulation formula becomes:

$$
\boxed{
S_T
= S_0
\exp\left[
\left(r - \frac{1}{2}\sigma^2\right)T
+ \sigma \sqrt{T}\,Z
\right]
}
$$

---

## 11. Interpretation

- $rT$: deterministic growth
- $-\frac{1}{2}\sigma^2 T$: volatility correction
- $\sigma \sqrt{T} Z$: random shock

This is the formula implemented in Monte Carlo option pricing.

In [8]:
# ==============================================================================
# 2. THE MONTE CARLO ENGINE (This is your Homework)
# ==============================================================================

def monte_carlo_pricer(S, K, T, r, sigma, simulations=10000):
    """
    Prices a European Call Option using Monte Carlo Simulation.
    
    Parameters:
    -----------
    S : float : Spot Price (Current Stock Price)
    K : float : Strike Price
    T : float : Time to Expiry (in years)
    r : float : Risk-free interest rate (e.g., 0.05)
    sigma : float : Volatility (standard deviation of log returns)
    simulations : int : Number of paths to simulate (default 10,000)
    
    Returns:
    --------
    float : Estimated Option Price
    """
    
    # --------------------------------------------------------------------------
    # STEP 1: GENERATE RANDOMNESS (Z)
    # --------------------------------------------------------------------------
    # Goal: Create an array of random numbers from a standard normal distribution.
    # The array should have a length equal to 'simulations'.
    # Hint: np.random.standard_normal(...)
    
    # z = ... (YOUR CODE HERE)
    
    z = np.random.standard_normal(simulations)
    
    
    
    # --------------------------------------------------------------------------
    # STEP 2: SIMULATE TERMINAL PRICES (S_T)
    # --------------------------------------------------------------------------
    # Goal: Implement the Geometric Brownian Motion solution.
    # Formula: S_T = S * exp( (r - 0.5 * sigma^2) * T + sigma * sqrt(T) * z )
    # Hint: Use np.exp() and np.sqrt()
    
    # ST = ... (YOUR CODE HERE)
    
    ST1 = S*np.exp((r-.5*sigma**2)*T+sigma*np.sqrt(T)*z)
    ST2 = S*np.exp((r-.5*sigma**2)*T+sigma*np.sqrt(T)*-z)
    

    
    # --------------------------------------------------------------------------
    # STEP 3: CALCULATE PAYOFF
    # --------------------------------------------------------------------------
    # Goal: Calculate the payoff for a Call Option.
    # Logic: Max(S_T - K, 0)
    # Hint: np.maximum(array, 0) is faster than a loop.
    
    # payoffs = ... (YOUR CODE HERE)
    payoffs1 = np.maximum(ST1-K,0)
    payoffs2 = np.maximum(ST2-K,0)
    
    payoffs = (payoffs1 + payoffs2)/2

    
    # --------------------------------------------------------------------------
    # STEP 4: DISCOUNT TO PRESENT VALUE
    # --------------------------------------------------------------------------
    # Goal: Average the payoffs and discount them back to today.
    # Formula: mean(payoffs) * exp(-r * T)
    
    # price = ... (YOUR CODE HERE)
    price = np.mean(payoffs)*np.exp(-r*T)
    
    return price

In [9]:
# ==============================================================================
# 3. MAIN EXECUTION LOOP
# ==============================================================================

def run_valuation(ticker):
    print(f"\n--- VALUING OPTIONS FOR {ticker} ---")
    
    # 1. Get Data
    data = get_market_data(ticker)
    S = data['spot_price']
    df = data['calls_df']
    
    if df.empty:
        print("No option data found.")
        return

    # 2. Get Inputs
    sigma = calculate_historical_volatility(data['historical_df'])
    r = get_risk_free_rate()
    
    print(f"Spot Price (S): ${S:.2f}")
    print(f"Volatility (σ): {sigma:.2%}")
    print(f"Risk-Free Rate (r): {r:.2%}")
    
    # 3. Calculate Time (T)
    df['T'] = df['expirationDate'].apply(calculate_time_to_expiry)
    
    # 4. Apply Monte Carlo Pricing
    print("\nRunning Monte Carlo simulations... (this might take a few seconds)")
    
    # We use a lambda function to apply your pricer to every row
    df['MC_Price'] = df.apply(
        lambda row: monte_carlo_pricer(S, row['strike'], row['T'], r, sigma), 
        axis=1
    )
    
    # 5. Display Results (Top 10 closest to money)
    first_expiry = df['expirationDate'].unique()[0]
    subset = df[df['expirationDate'] == first_expiry].copy()
    
    # Filter for "Near the Money" options (Strike close to Spot)
    subset['dist'] = abs(subset['strike'] - S)
    near_the_money = subset.sort_values('dist').head(10).sort_values('strike')
    
    print(f"\nResults for Expiry: {first_expiry}")
    print(near_the_money[['strike', 'lastPrice', 'MC_Price']])

if __name__ == "__main__":
    run_valuation("NVDA") # <--- Change Ticker Here


--- VALUING OPTIONS FOR NVDA ---
Fetching option chains for NVDA...
Spot Price (S): $174.57
Volatility (σ): 49.90%
Risk-Free Rate (r): 3.52%

Running Monte Carlo simulations... (this might take a few seconds)

Results for Expiry: 2025-12-19
     strike  lastPrice  MC_Price
227   171.0       4.35  4.132411
228   172.0       3.55  3.382124
229   172.5       3.15  3.049266
230   173.0       2.80  2.742481
231   174.0       2.09  2.124292
232   175.0       1.48  1.622841
233   176.0       0.97  1.211657
234   177.0       0.61  0.882935
235   177.5       0.48  0.736379
236   178.0       0.37  0.639129
